In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
! unzip /kaggle/input/numerical-image-dataset

In [1]:
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.5 ## only use 50 % of the gpu
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

import tensorflow as tf

device_name = tf.test.gpu_device_name()

if "GPU" not in device_name:
    print("GPU device not found")
    
print('Found GPU at: {}'.format(device_name))

print("GPU", "available (YESS!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")

2024-02-29 17:47:55.680551: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 17:47:55.680722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 17:47:55.846471: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found GPU at: /device:GPU:0
GPU available (YESS!!!!)


###  Importing Necessary Library 

In [2]:
import numpy as np
from glob import glob
from tensorflow.keras.layers import (Input, Lambda, Dense, Flatten, Conv2D)
from tensorflow.keras.models import (Model, Sequential)
from tensorflow.keras.applications.mobilenet_v2 import ( MobileNetV2, preprocess_input)
from tensorflow.keras.preprocessing.image import (ImageDataGenerator, load_img, array_to_img, img_to_array)
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input

In [3]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/kaggle/input/numerical-image-dataset/dataset/train'
valid_path = '/kaggle/input/numerical-image-dataset/dataset/val'
test_path =  "/kaggle/input/numerical-image-dataset/dataset/test"

In [4]:
# Import the MobileNet library as shown below and add preprocessing layer to the front of MobileNet
# Here we will be using imagenet weights

mobile_net  = MobileNetV2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False,
                         pooling='avg')

# resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
# mobile_net.summary()

In [6]:
print("No Of The Layer In MObileNet :--> ", len(mobile_net.layers))

No Of The Layer In MObileNet :-->  155


In [7]:
  # useful for getting number of output classes
folders = glob('/kaggle/input/numerical-image-dataset/dataset/train/*')
print("total no of the folder in the train dataset :-->", len(folders))

total no of the folder in the train dataset :--> 100


### Data Preprocessing

In [8]:
# Use the Image Data Generator to import the images from the dataset

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Define data generators for training and validation data
train_datagen = ImageDataGenerator(
    rescale=1./255,            # Rescale pixel values to the range [0, 1]
    preprocessing_function=preprocess_input,
    rotation_range=40,         # Randomly rotate images by up to 40 degrees
    width_shift_range=0.3,     # Randomly shift images horizontally by up to 30% of the width
    height_shift_range=0.3,    # Randomly shift images vertically by up to 30% of the height
    shear_range=0.3,           # Shear intensity
    zoom_range=0.3,            # Randomly zoom images by up to 30%
    horizontal_flip=True,      # Randomly flip images horizontally
    vertical_flip=True,        # Randomly flip images vertically
    brightness_range=[0.6, 1.4],  # Randomly adjust brightness
    channel_shift_range=100.0,  # Randomly shift channels
    fill_mode='nearest',       # Strategy for filling in newly created pixels
    validation_split=0.2       # Fraction of training data to use for validation
)

test_datagen = ImageDataGenerator(rescale=1./255,   # Rescale pixel values for the test set
                                  preprocessing_function=preprocess_input)

In [9]:
# Make sure to provide the same target size as initialied for the image size

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

validation_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 25300 images belonging to 100 classes.
Found 3200 images belonging to 100 classes.


In [10]:
print("LIST OF THE CLASS NAME  ARE :-", list(training_set.class_indices.keys()))

LIST OF THE CLASS NAME  ARE :- ['1', '10', '100', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']


In [11]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(mobile_net.layers))

# Fine-tune from this layer onwards
fine_tune_at = 150

# Freeze all the layers before the `fine_tune_at` layer
for layer in mobile_net.layers[:fine_tune_at]:
    layer.trainable =  False

Number of layers in the base model:  155


In [12]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

#learning_rate reduce module
lr_reduce = ReduceLROnPlateau('val_loss', patience=4,
                                              factor=0.5, min_lr=1e-6)

# Stop early if model doesn't improve after n epochs
early_stopper = EarlyStopping(monitor='val_loss', patience=4,
                              verbose=0, restore_best_weights=True)

In [13]:
from tensorflow.keras.layers import (
    LeakyReLU, PReLU, ReLU, Conv2D, MaxPooling2D, Dropout, BatchNormalization
)

model = Sequential()

model.add(mobile_net)
model.add(Dense(256, activation = 'relu'))
model.add(Dense(len(folders), activation  = 'softmax'))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ ?                      │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 412,800 (1.57 MB)

 Non-trainable params: 1,845,184 (7.04 MB)

In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ ?                      │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 412,800 (1.57 MB)

 Non-trainable params: 1,845,184 (7.04 MB)

In [15]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
history_fine = model.fit(training_set,
                         batch_size = 32,
                         epochs=500,
                         validation_data=validation_set,
                         callbacks=[lr_reduce, early_stopper],
                         verbose=1, shuffle=True)

Epoch 1/500


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  1/791 ━━━━━━━━━━━━━━━━━━━━ 4:23:58 20s/step - accuracy: 0.0000e+00 - loss: 4.7024

I0000 00:00:1709228913.010090     104 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1709228913.042122     104 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


594/791 ━━━━━━━━━━━━━━━━━━━━ 1:45 533ms/step - accuracy: 0.0083 - loss: 4.6278

W0000 00:00:1709229229.045027     107 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


790/791 ━━━━━━━━━━━━━━━━━━━━ 0s 527ms/step - accuracy: 0.0082 - loss: 4.6234

W0000 00:00:1709229332.515388     104 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


791/791 ━━━━━━━━━━━━━━━━━━━━ 457s 553ms/step - accuracy: 0.0082 - loss: 4.6233 - val_accuracy: 0.0100 - val_loss: 4.6054 - learning_rate: 0.0010
Epoch 2/500
791/791 ━━━━━━━━━━━━━━━━━━━━ 330s 413ms/step - accuracy: 0.0084 - loss: 4.6056 - val_accuracy: 0.0100 - val_loss: 4.6055 - learning_rate: 0.0010
Epoch 3/500
791/791 ━━━━━━━━━━━━━━━━━━━━ 323s 405ms/step - accuracy: 0.0071 - loss: 4.6056 - val_accuracy: 0.0100 - val_loss: 4.6052 - learning_rate: 0.0010
Epoch 4/500
648/791 ━━━━━━━━━━━━━━━━━━━━ 56s 398ms/step - accuracy: 0.0083 - loss: 4.6056

In [ ]:
import matplotlib.pyplot as plt
# plot the loss
plt.plot(history_fine.history['loss'], label='train loss')
plt.plot(history_fine.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(history_fine.history['accuracy'], label='train acc')
plt.plot(history_fine.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
# save it as a h5 file


from tensorflow.keras.models import load_model

model.save('resnet.h5')

In [ ]:
y_pred = np.argmax(model.predict(validation_set), axis =1)

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

model = load_model('resnet.h5')

In [ ]:
img=image.load_img('/kaggle/input/numerical-image-dataset/dataset/test/1/image_103.png',target_size=(224,224))


In [ ]:
x=image.img_to_array(img)
x=x/255

In [ ]:
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

In [ ]:
model.predict(img_data)

In [ ]:
a=np.argmax(model.predict(img_data), axis=1)
a